# Caesium-133 D Line

Rubidium is a heavy atom with hyperfine structure. Modelling of the Rb<sup>85</sup> D line is shown in this notebook using the `LASED` library. The calculations will be compared with J.Pursehouse's PhD thesis as he modelled the same system.

In [1]:
%run "base-LASED/LASED/laser_atom_system.py"
%run "base-LASED/LASED/time_evolution.py"
%run "base-LASED/LASED/generate_sub_states.py"

import plotly.graph_objects as go
import time

The data for Rubidium can be accessed [here](https://ethos.bl.uk/OrderDetails.do?uin=uk.bl.ethos.740344). The 5$^2$S$_{1/2}$ to the 5$^2$P$_{3/2}$ transition will be modelled here with F' = 3 to the F = 4 level as the resonant transition and all other terms will be detuned from this resonance. A level diagram of the system is shown below:

In [2]:
from IPython.display import Image
#Image(filename = "LevelDiagrams/Rb-85-5S1_2to5P3_2.jpg")

In [3]:
# 6^2S_{1/2} -> 6^2P_{3/2}
wavelength_cs = 852.356e-9  # Wavelength in nm
w_e = te.angularFreq(wavelength_cs)
tau_cs = 30.473 # in ns

I_cs = 7/2  # Isospin for sodium 

# Energy Splittings
w1 = 9.192631770*2*PI # Splitting of 6^2S_{1/2}(F' = 3) -> (F' = 4) in Grad/s (Exact due to definition of the second)
w2 = 0.15121*2*PI  # Splitting between 6^2P_{3/2} F = 2 and F = 3 in Grad/s
w3 = 0.20124*2*PI  # Splitting between 6^2P_{3/2} F = 3 and F = 4 in Grad/s
w4 = 0.251*2*PI  # Splitting between 6^2P_{3/2} F = 4 and F = 5 in Grad/s

# Detunings
w_Fp3 = -1*w1
w_F2 = w_e-(w4+w3+w2)
w_F3 = w_e-(w4+w3)
w_F4 = w_e-w4
w_F5 = w_e

# Create states 
# 6^2S_{1/2}
Fp3 = generateSubStates(label_from = 1, w = w_Fp3, L = 0, S = 1/2, I = I_cs, F = 3)
Fp4 = generateSubStates(label_from = 8, w = 0, L = 0, S = 1/2, I = I_cs, F = 4)

# 5^2P_{3/2}
F2 = generateSubStates(label_from = 17, w = w_F2, L = 1, S = 1/2, I = I_cs, F = 2)
F3 = generateSubStates(label_from = 22, w = w_F3, L = 1, S = 1/2, I = I_cs, F = 3)
F4 = generateSubStates(label_from = 29, w = w_F4, L = 1, S = 1/2, I = I_cs, F = 4)
F5 = generateSubStates(label_from = 38, w = w_F5, L = 1, S = 1/2, I = I_cs, F = 5)

# Declare excited and ground states
G_cs = Fp3 + Fp4
E_cs = F2 + F3 + F4 + F5

# Laser parameters
intensity_cs = 50 # mW/mm^-2
Q_cs = [0]
Q_decay = [1, 0, -1]

# Simulation parameters
start_time = 0
stop_time = 500 # in ns
time_steps = 501
time_cs = np.linspace(start_time, stop_time, time_steps)

Create a `LaserAtomSystem` object and time evolve the system using `timeEvolution()`

In [ ]:
cs_system = LaserAtomSystem(E_cs, G_cs, tau_cs, Q_cs, wavelength_cs, laser_intensity = intensity_cs)
tic = time.perf_counter()
cs_system.timeEvolution(time_cs)
toc = time.perf_counter()
print(f"The code finished in {toc-tic:0.4f} seconds")

Populating ground states equally as the initial condition.


Now plot the results

In [ ]:
rho_to_plot = [ [abs(rho) for rho in cs_system.Rho_t(s, s)] for s in F5]

fig_csF5 = go.Figure()

for i, rho in enumerate(rho_to_plot):
    fig_csF5.add_trace(go.Scatter(x = time_cs, 
                                y = rho,
                                name = f"m_F = {F5[i].m}",
                               mode = 'lines'))

fig_rbF4.update_layout(title = "<sup>133</sup>Caesium 6sup>2</sup>S<sub>1/2</sub> to 6<sup>2</sup>P<sub>3/2</sub> π-Excitation: Time Evolution of Population in the F = 5 Sub-states",
                 xaxis_title = "Time (ns)",
                 yaxis_title = "Population",
                font = dict(
                    size = 11))
fig_rbF4.write_image("CsF=5I=50.pdf")
fig_rbF4.show()